In [0]:
!nvidia-smi

In [0]:
## Essential Libraries.

import numpy as np

import torch

import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

In [0]:
## Hyperparameters.

FLAGS = {
"batch_size": 128,
"learning_rate" : 0.001,
"n_epoch1": 30,
"eps1": 0.3,
"n_epoch2": 5,
"eps2": 0.4}

In [0]:
## Reading Dataset.

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=FLAGS["batch_size"],
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=10000,
                                         shuffle=False, num_workers=1)

In [0]:
## Model.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (5, 5), stride=1)
        self.pool = nn.MaxPool2d((2, 2), stride=2)
        self.conv2 = nn.Conv2d(32, 64, (5, 5), stride=1)
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [0]:
net = Net()

In [0]:
## Loss and optimizer.

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=FLAGS["learning_rate"])

In [0]:
# GPU!

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

In [0]:
## PGD attack.

def pgd_attack(model, images, labels, eps=FLAGS["eps1"], alpha=0.01, iters=40):

    # iters = int((2.5 * eps) / alpha)
    
    images = images.to(device)
    labels = labels.to(device)

    loss = torch.nn.CrossEntropyLoss()

    ori_images = images
    
    images = images + torch.zeros_like(images).uniform_(-eps, eps)
    images = torch.clamp(images, min=0, max=1)

    for i in range(iters):
        
        images.requires_grad = True

        outputs = model(images)

        model.zero_grad()

        cost = loss(outputs, labels).to(device)
        cost.backward()

        adv_images = images + alpha*images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()
        
    return images

In [0]:
net.train()

num_epchos = FLAGS["n_epoch1"]

for epoch in range(num_epchos):
    
    steps = 0
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        
        adv = pgd_attack(net, inputs, labels)

        outputs = net(adv)
    
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        steps += 1
        running_loss += loss.item()

    # print statistics
    print('%d loss: %.5f' % (epoch + 1, running_loss / steps))

print('Finished Training')

In [0]:
net.train()

num_epchos = FLAGS["n_epoch2"]

for epoch in range(num_epchos):
    
    steps = 0
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        
        adv = pgd_attack(net, inputs, labels, eps=FLAGS["eps2"], alpha=0.005, iters=200)

        outputs = net(adv)
    
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        steps += 1
        running_loss += loss.item()

    # print statistics
    print('%d loss: %.5f' % (epoch + 1, running_loss / steps))

print('Finished Training')

In [0]:
## Saving the model.

net.eval()

PATH = './mnist-' + str(FLAGS["eps1"]) + '-' + str(FLAGS["eps2"]) + '.pth'
torch.save(net.state_dict(), PATH)

In [0]:
net.eval()

In [0]:
## Natural accuracy.

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (100 * correct / total))

In [0]:
## Adversarial accuracy.

correct = 0
total = 0
for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    adv = pgd_attack(net, images, labels, eps=0.4)
    outputs = net(adv)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print('Adversarial Accuracy of the network on the 10000 test images: %f %%' % (100 * correct / total))